## 1. requests

In [53]:
import requests as r
import os
import pandas as pd
import time
import json


In [51]:
# create a directory it it doesn't exist
def createFolder(dir_name):
    try:
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
    except OSError:
        print ('Error: Creating directory. ' +  dir_name)

In [36]:
key1 = "" # Your bing API key

In [37]:
#set your path
base_path = "" # Your base_path
os.chdir(base_path)

### 1. function to make image-files for one keyword

In [75]:
def make_imgs(item, num, apikey):
    # save start time to measure performance time
    start = time.time()
    # make a list to save error log
    error_log = []
    
    # set the directory
    os.chdir("./bing_img") # your designated directory
    createFolder(item)
    os.chdir(f"./{item}")
    
    url = "https://api.bing.microsoft.com/v7.0/images/search"
    headers = {
              "Ocp-Apim-Subscription-Key" : key1
              }
    
    # you can manage many query parameters. check out the documentation
    # https://docs.microsoft.com/en-us/bing/search-apis/bing-image-search/reference/query-parameters
    
    param = {"q" : item,
          "count" : num, #choose num of results from 1 to 150
          "imageType" : "Photo",
          "imageContent" : "Face",
          "minHeight" : 256, # set minimum height(pixels)
          "safeSearch" : "strict", # filter adult contents
          "color" : "ColorOnly"
          }
    s = r.Session()
    test = s.get(url,headers=headers, params=param).json()
    
    # download images and labels using link result
    for i in range(0,len(test.get("value"))):
          file_name = item+"_"+str(i)
          print(file_name)
          print(test.get('value')[i].get('contentUrl'))
          json_file_name = f"{file_name}.json"
          file_path = os.getcwd()+'/'+json_file_name
          img_file_name = f"{file_name}.{test.get('value')[i].get('encodingFormat')}"
          #print(img_file_name)
          #set headers to avoid 403 forbidden error
          headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)Chrome/91.0.4472.77 Safari/537.36'}
            
          img_r = r.get(test.get('value')[i].get('contentUrl'), headers=headers, timeout=10, stream=True)
          with open (img_file_name,'wb') as f:
            try:
                  f.write(img_r.content)
                  
            except Exception as e:
                  print(e)
                  error_log.append(f"{img_file_name}: {e} ")   
                  pass       
      
            data = {}
            data[file_name] = []
            data[file_name].append({
                  "contentUrl" : test.get('value')[i].get('contentUrl'),
                  "datePublished" : test.get('value')[i].get('datePublished'),
                  "encodingFormat" : test.get('value')[i].get('encodingFormat'),
                  "height" : test.get('value')[i].get('height'),
                  "hostPageUrl" : test.get('value')[i].get('hostPageUrl'),
                  "imageId" : test.get('value')[i].get('imageId'),
                  "width" : test.get('value')[i].get('width')
            })
            with open(file_path,'w') as outfile:
                  json.dump(data,outfile)


        
    # get back to the original path
    os.chdir("../..")
    time_taken = time.time()-start
    return time_taken, error_log

In [76]:
make_imgs("Kamala Harris", 10, key1)

Kamala Harris_0
https://it.insideover.com/wp-content/uploads/2019/12/LP_10733056-e1575976287635.jpg
Kamala Harris_1
https://static.timesofisrael.com/blogs/uploads/2020/11/Kamala_Harris_April_2019.jpg
Kamala Harris_2
https://www.biography.com/.image/ar_1:1%2Cc_fill%2Ccs_srgb%2Cfl_progressive%2Cq_auto:good%2Cw_1200/MTc0NjMzMzkyODAzOTQ4NDg5/kamala-harris_500x500_gettyimages-1158742275.jpg
Kamala Harris_3
https://www.kerknet.be/sites/default/files/styles/width_640/public/harris.jpg?itok=XWgiTtBX
Kamala Harris_4
https://conservativemodern.com/wp-content/uploads/2020/11/kamala-harris-38.jpg
Kamala Harris_5
https://2.bp.blogspot.com/-9ulH01DNg_c/W5cQq8TTJbI/AAAAAAABQcs/8P_lHca3YZQ8QQaVVI4V_oYdUqYHXUs2QCLcBGAs/s1600/Kamala-Harris2.jpg
Kamala Harris_6
http://www1.pictures.zimbio.com/gi/Sen+Kamala+Harris+Conversation+KZXPGOt5BFCx.jpg
Kamala Harris_7
https://pyxis.nymag.com/v1/imgs/e56/65d/0ea07cfb4c77b9d2f11cc7a04c660ecb65-kamala-harris-debate.rsquare.w700.jpg
Kamala Harris_8
https://api.time.co

(19.991685152053833, [])

### 2. function to make image-file lists for several keywords

In [77]:
# 쿼리를 요청할 검색어 리스트 만들거나 불러오기
name_list = ['Ruth Bader Ginsburg', 'Elizabeth Warren', 'Nancy Pelosi']

In [78]:
def make_img_list(lst, num, apikey):
  # 시작 시간 저장
  start = time.time()
  # 에러 로그 저장을 위한 리스트 만들기
  error_log = []
  # 주어진 리스트에 있는 모든 이름 검색
  for item in lst:

    # 폴더 만들고 경로 이동하기
    os.chdir(base_path+"/bing_img") # your designated directory
    createFolder(item)
    os.chdir(f"./{item}")

    url = "https://api.bing.microsoft.com/v7.0/images/search"
    headers = {
              "Ocp-Apim-Subscription-Key" : key1
              }
    param = {"q" : item,
          "count" : num,
          "imageType" : "Photo",
          "imageContent" : "Face",
          "minHeight" : 256,
          "safeSearch" : "strict",
          "color" : "ColorOnly"
          }
    s = r.Session()
    test = s.get(url,headers=headers, params=param).json()
    # 검색결과 링크에서 이미지 저장하기

    for i in range(0,len(test.get("value"))):
        file_name = item+"_"+str(i)
        print(file_name)
        print(test.get('value')[i].get('contentUrl'))
        json_file_name = f"{file_name}.json"
        file_path = os.getcwd()+'/'+json_file_name
        img_file_name = f"{file_name}.{test.get('value')[i].get('encodingFormat')}"
        #print(img_file_name)
        
        #set headers to avoid 403 forbidden error
        headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)Chrome/91.0.4472.77 Safari/537.36'}
        
        img_r = r.get(test.get('value')[i].get('contentUrl'), headers=headers, timeout=10, stream=True)
        with open (img_file_name,'wb') as f:
            try:
                  f.write(img_r.content)
                  
            except Exception as e:
                  print(e)
                  error_log.append(f"{img_file_name}: {e} ")   
                  pass       
      
            data = {}
            data[file_name] = []
            data[file_name].append({
                  "contentUrl" : test.get('value')[i].get('contentUrl'),
                  "datePublished" : test.get('value')[i].get('datePublished'),
                  "encodingFormat" : test.get('value')[i].get('encodingFormat'),
                  "height" : test.get('value')[i].get('height'),
                  "hostPageUrl" : test.get('value')[i].get('hostPageUrl'),
                  "imageId" : test.get('value')[i].get('imageId'),
                  "width" : test.get('value')[i].get('width')
            })
            with open(file_path,'w') as outfile:
                  json.dump(data,outfile)
        
    # get back to base_path
    os.chdir("../..")
    
  # measure performance time
  time_taken = time.time()-start
  return time_taken, error_log

In [79]:
make_img_list(name_list, 3 ,key1)

Ruth Bader Ginsburg_0
https://media.nbcbayarea.com/2019/09/ruth-bader-ginsburg-1.jpg?fit=1786%2C1727
Ruth Bader Ginsburg_1
http://wpitechnews.com/wp-content/uploads/2020/09/906px-Ruth_Bader_Ginsburg_2016_portrait_cropped.jpg
Ruth Bader Ginsburg_2
https://www.biography.com/.image/t_share/MTE5NTU2MzE2MjMwNTUxMDUx/ruth-bader-ginsburg-9312041-1-402.jpg
Elizabeth Warren_0
https://townsquare.media/site/918/files/2018/07/Elizabeth-Warren.jpg?w=1200
Elizabeth Warren_1
https://mediaproxy.salon.com/width/1200/https://media.salon.com/2013/08/elizabeth_warren8.jpg
Elizabeth Warren_2
https://starsinformer.com/wp-content/uploads/2017/08/Elizabeth-Warren-5.jpg
Nancy Pelosi_0
https://www.famousplasticsurgeries.com/wp-content/uploads/2020/12/Nancy-Pelosi-Botox-Nose-Job-Lips-Plastic-Surgery-Rumors-819x1024.jpg
Nancy Pelosi_1
https://www.theamericanmirror.com/wp-content/uploads/2019/12/Nancy-Pelosi-bandage-e1575992619662.jpg
Nancy Pelosi_2
https://www.the-sun.com/wp-content/uploads/sites/6/2020/04/NINTCH

(14.829231023788452, [])